**WRITER'S IDENTITY**
- AZIMIL GANI ALAM
- Ph.D Student
- Dept. Energy & Process Tech - NTNU


**NOTE :**
- This Python File runs in GoogleColab
- This work uses **All Data**
- we want to combine all data results from CSV models become process-ready for data visualization

# **Library Import**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statistics
import re
import os
from datetime import datetime, timedelta
import pytz

import warnings
warnings.filterwarnings("ignore")
import psychrolib
from pythermalcomfort.models import pmv_ppd
from pythermalcomfort.utilities import v_relative, clo_dynamic

In [2]:
def datetime_indexer(df):
    df['datetime'] = pd.to_datetime((df['date'] + ' ' + df['time']), format='%Y-%m-%d %H:%M')
    df.set_index(['datetime'], inplace = True)

def datetime_indexer_noindex(df):
    df['datetime'] = pd.to_datetime((df['date'] + ' ' + df['time']), format='%Y-%m-%d %H:%M')
    #df.set_index(['datetime'], inplace = True)

def datedayhour_creator(df) :
  df.reset_index( inplace = True)
  datechange = pd.to_datetime(df['datetime'])
  df['date'] = datechange.dt.strftime('%Y-%m-%d')
  df['day']  = datechange.dt.dayofweek
  df['hour'] = datechange.dt.hour

def datedayhour_creator_noindex(df) :
  datechange = pd.to_datetime(df['datetime'])
  df['date'] = datechange.dt.strftime('%Y-%m-%d')
  df['day']  = datechange.dt.dayofweek
  df['hour'] = datechange.dt.hour

def date_selector(df,start_date,end_date) :
  df.reset_index( inplace = True)
  datechange = pd.to_datetime(df['datetime'])
  df['date'] = datechange.dt.strftime('%Y-%m-%d')
  df         = df[(df['date']>=start_date) & (df['date']<=end_date)]
  df         = df.drop('date', axis=1).set_index('datetime')
  

def pmv_ppd_generator(df):
  tdb = df['temp_a'].values
  tr = tdb
  vel = 0.05
  rh = df['rh_a'].values
  met = 1.2
  clo = 0.67

  v_rel = v_relative(vel, met)
  clo_d = clo_dynamic(clo, met)
  results = pmv_ppd( tdb=tdb, tr=tr ,vr=v_rel, rh= rh, met=met, clo=clo_d,standard= 'iso', units= 'SI')
  df['pmv'] = results['pmv']
  df['ppd'] = results['ppd']/100

In [3]:
def quantitative_niluapp(DF3):
  # DF3['feel_air0'] = 0
  # DF3['feel_air1'] = 0
  # DF3['feel_air2'] = 0
  # DF3['feel_air3'] = 0
  # DF3['feel_air4'] = 0
  # DF3['feel_temp0'] = 0
  # DF3['feel_temp1'] = 0
  # DF3['feel_temp2'] = 0
  # DF3['feel_temp3'] = 0
  # DF3['feel_temp4'] = 0
  # DF3['feel_bright0'] = 0
  # DF3['feel_bright1'] = 0
  # DF3['feel_bright2'] = 0
  # DF3['feel_bright3'] = 0
  # DF3['feel_bright4'] = 0
  # DF3['feel_health0'] = 0
  # DF3['feel_health1'] = 0
  # DF3['feel_health2'] = 0
  # DF3['feel_health3'] = 0
  # DF3['feel_health4'] = 0
  # DF3['feel_noise0'] = 0
  # DF3['feel_noise1'] = 0
  # DF3['feel_noise2'] = 0
  # DF3['feel_noise3'] = 0
  # DF3['feel_noise4'] = 0
  DF3.loc[DF3['feel_air'] == 0, 'feel_air0'] = 1
  DF3.loc[DF3['feel_air'] == 1, 'feel_air1'] = 1
  DF3.loc[DF3['feel_air'] == 2, 'feel_air2'] = 1
  DF3.loc[DF3['feel_air'] == 3, 'feel_air3'] = 1
  DF3.loc[DF3['feel_air'] == 4, 'feel_air4'] = 1
  DF3.loc[DF3['feel_temp'] == 0, 'feel_temp0'] = 1
  DF3.loc[DF3['feel_temp'] == 1, 'feel_temp1'] = 1
  DF3.loc[DF3['feel_temp'] == 2, 'feel_temp2'] = 1
  DF3.loc[DF3['feel_temp'] == 3, 'feel_temp3'] = 1
  DF3.loc[DF3['feel_temp'] == 4, 'feel_temp4'] = 1
  DF3.loc[DF3['feel_bright'] == 0, 'feel_bright0'] = 1
  DF3.loc[DF3['feel_bright'] == 1, 'feel_bright1'] = 1
  DF3.loc[DF3['feel_bright'] == 2, 'feel_bright2'] = 1
  DF3.loc[DF3['feel_bright'] == 3, 'feel_bright3'] = 1
  DF3.loc[DF3['feel_bright'] == 4, 'feel_bright4'] = 1
  DF3.loc[DF3['feel_health'] == 0, 'feel_health0'] = 1
  DF3.loc[DF3['feel_health'] == 1, 'feel_health1'] = 1
  DF3.loc[DF3['feel_health'] == 2, 'feel_health2'] = 1
  DF3.loc[DF3['feel_health'] == 3, 'feel_health3'] = 1
  DF3.loc[DF3['feel_health'] == 4, 'feel_health4'] = 1
  DF3.loc[DF3['feel_noise'] == 0, 'feel_noise0'] = 1
  DF3.loc[DF3['feel_noise'] == 1, 'feel_noise1'] = 1
  DF3.loc[DF3['feel_noise'] == 2, 'feel_noise2'] = 1
  DF3.loc[DF3['feel_noise'] == 3, 'feel_noise3'] = 1
  DF3.loc[DF3['feel_noise'] == 4, 'feel_noise4'] = 1

# Default  Names & Setting

In [15]:
# Folder Naming
folder_raw     = 'data-raw'
folder_ready   = 'data-ready'
folder_year    = 'all years'
folder_project = 'Project'
folder_export  = 'export'
folder_is      = 'niluapp'
data_for1      = 'combined panel'
data_for2      = 'combined longitudinal'
data_for3      = 'calibration'
platform_directory = 'G:/My Drive/Colab Notebooks'   #Computer NILU
# platform_directory = 'C:/Users/azimilga/My Drive/Colab Notebooks'   #laptop NTNU

In [16]:
#filtering rows data regarding to desired specific time
start_time = '06:00'
end_time   = '13:00'
start_date = '023-03-06'
end_date   =  '2023-05-30'

spec_date = pd.to_datetime('today').strftime("%Y-%m-%d")
#name_date = pd.to_datetime('today').strftime("%y%m%d")
date_format_show =  '%Y-%m-%d'
time_format_show =  '%H:%M'
start_day  = 0
end_day    = 4
start_hour = 6
end_hour   = 13

# tell resample data
time_series_resample = '10min'     #for Processor 1
survey_resample      = '10min'     #for Processor 2
desample_option      = '10min'
minutely_resample    = '1min'

In [11]:
col_header = ['title', 'sessionid', 'date', 'time' ]
response_counting = ['responses']
is_col_main  =  ['feel_air','feel_temp','feel_health','feel_bright','feel_noise']
is_col_main1  = ['feel_air1','feel_temp1','feel_health1','feel_bright1'] #,'feel_noise1'
is_col_main2  = ['feel_air2','feel_temp2','feel_health2','feel_bright2'] #,'feel_noise2'
is_col_main3  = ['feel_air3','feel_temp3','feel_health3','feel_bright3'] #,'feel_noise3'
is_col_main4  = ['feel_air4','feel_temp4','feel_health4','feel_bright4'] #,'feel_noise4'

is_main_air    = ['feel_air1', 'feel_air2', 'feel_air3', 'feel_air4']
is_main_temp   = ['feel_temp1', 'feel_temp2', 'feel_temp3', 'feel_temp4']
is_main_bright = ['feel_bright1', 'feel_bright2', 'feel_bright3', 'feel_bright4']
is_main_health = ['feel_health1', 'feel_health2', 'feel_health3', 'feel_health4']
is_main_noise  = ['feel_noise1', 'feel_noise2', 'feel_noise3', 'feel_noise4']

is_col_air   = ['air_smell','air_heavy','air_dry','air_dust','air_electshock']
is_col_temp  = ['temp_coldhot','temp_draw','temp_coldfloor','temp_heatsun','temp_heater' ]
is_col_bright = ['bright_sun','bright_lamp_hi','bright_lamp_low']
is_col_health  = ['health_head','health_cough','health_tired','health_dryskin']
is_col_multi  = ['feel_air','feel_temp','feel_health','feel_bright','feel_noise','temp_coldhot']
is_col_subquestion = is_col_air+ is_col_temp + is_col_bright + is_col_health
pd_col       = ['pd_air', 'pd_temp', 'pd_health', 'pd_bright', 'pd_noise']
columns_formation   = col_header + is_col_main + is_col_air + is_col_temp + is_col_bright + is_col_health

school_col_name   = 'school_id'
room_col_name     = 'room_id'
date_name         = 'date'
time_name         = 'time'
manuf_col_name    = 'manuf_id'
instr_col_name    = 'instr_id'
serial_col_name   = 'serial_id'

#Give a name for new column in purpose to count how many response / hour or response / day
response_counting = 'responses'

#columns_formation = ['title', 'sessionid', 'date', 'time' , 'Room Air', 'Room Temperature', 'Room Lighting', 'Health Feeling', 'Room Noice', 'BAD SMELL' , 'BAD HEAVY/AIR', 'DRY AIR', 'DUST AND DIRT', 'ELECTRIC SHOCK', '(-COLD) OR HOT', 'DRAWING COLD AIR', 'COLD ON THE FLOOR', 'MUCH HEAT FROM SUNSHINE', 'MUCH HEAT FROM FURNACES', 'LIGHT FROM THE SUN', 'BRIGHT LIGHT CEILING LAMPS', 'WEAK LIGHT CEILING LAMPS', 'HEADACHE', 'COUGH/SHORENESS', 'TIRED/UNCONCENTRATED', 'DRY EYES/HANDS']


word_indoor   = 'Indoor '
word_outdoor  = 'Outdoor ' 
temp_v_label  = word_indoor+ 'Temp. by Vent system (*C)'
co2_v_label   = word_indoor+ 'CO2 level by Vent System (ppm)'
vent_v_label  = word_indoor+ 'Supply Air (CMH)'
temp_a_label  = word_indoor+ 'Temp. by add. sensors (*C)'
rh_a_label    = word_indoor+ 'Relative Humidity (%)'
co2_a_label   = word_indoor+ 'CO2 level by add. sensors (ppm)'
voc_a_label   = word_indoor+ 'VOC contaminant level (ppb)'
bright_a_label= word_indoor+ 'Luminous Intensity (%)'
sound_a_label = word_indoor+ 'Sound Pressure Intensity (Pa)'
pm25_a_label  = word_indoor+ 'PM2.5 density (ug/m3)'
pm1_a_label   = word_indoor+ 'PM1.0 density (ug/m3)'
rdn_a_label   = word_indoor+ 'Radon Level (Bq/m3)'
press_a_label = word_indoor+ 'Air pressure (hPa)'
swc_f_label   = word_indoor+ 'Floor Heater -On/Off'
swc_r_label   = word_indoor+ 'Baseboard Heater-On/Off'
temp_f_label  = word_indoor+ 'Floor Temperature (*C)'
temp_o_label  = word_outdoor+ 'Temperature (*C)'
rh_o_label    = word_outdoor+ 'Relative Humidiy (%)'
winds_o_label = word_outdoor+ 'Wind Speed (m/s)'
sun_o_label   = word_outdoor+ 'Mean Global Radiation (W/m2)'
pm25_o_label  = word_outdoor+ 'PM2.5 density (ug/m3)'
pm10_o_label  = word_outdoor+ 'PM10 density (ug/m3)'

directory_path = platform_directory +  '/' + folder_project + '/' + folder_ready   +  '/'   +  folder_year  + '/'
co2_name          = 'co2_a'
temp_data_name    = 'temp_a'
rel_hum_name      = 'rh_a'
voc_name          = 'voc_a'
bright_name        = 'bright_a'
sound_name        = 'sound_a'
pm25_name         = 'pm2.5_a'
pm1_name          = 'pm1_a'
radon_name        = 'rdn_a'
press_name        = 'press_a'


airthingsPlus = [ co2_name,temp_data_name, rel_hum_name, press_name, 'enth_a', 'hura_a', voc_name,bright_name, radon_name, 'pmv', 'ppd']
airthingsPro  = [ co2_name,temp_data_name, rel_hum_name, press_name, 'enth_a', 'hura_a', voc_name,bright_name, sound_name, pm1_name, pm25_name,  'pmv', 'ppd']
weather_volda = [ 'temp_o', 'rh_o', 'winds_o', 'rain_o','enth_o', 'hura_o', 'press_o']
weather_oslo  = [ 'temp_o', 'rh_o', 'winds_o', 'rain_o','enth_o', 'hura_o', 'press_o', 'sun_o', 'pm2.5_o', 'pm10_o']
weather_extra = [ 'nox_o', 'no2_o', 'no_o']

Time Zone Changing (GMT to OSLO)

In [12]:
 #convert time format to datetime  : time_zone_converter (df, 'Zone Origin', 'Zone Destination')
def time_zone_converter(df, zone_origin, zone_destination):
  #df['datetime'] = pd.to_datetime((df['date'] + ' ' + df['time']), format='%Y-%m-%d %H:%M:%S')
  #df['DateTime'] = datechange.dt.strftime('%Y-%m-%d %H:%M:%S')

  df['datetime'] = pd.to_datetime(df['datetime'])
  df['date'] = df['datetime'].dt.strftime('%Y-%m-%d')

  # Define the timezones (GMT and Europe/Oslo)
  origin_timezone = pytz.timezone(zone_origin)
  destination_timezone = pytz.timezone(zone_destination)

    # Adjust the datetime to the Oslo timezone and account for daylight saving time
  df['adjusted datetime'] = df['datetime'].dt.tz_localize(origin_timezone, ambiguous='NaT', nonexistent='NaT').dt.tz_convert(destination_timezone)

  # Extract the adjusted time
  df['date'] = df['adjusted datetime'].dt.strftime('%Y-%m-%d')
  df['time'] = df['adjusted datetime'].dt.strftime('%H:%M')
  df['hour'] = df['adjusted datetime'].dt.hour
  df['datetime'] = df['adjusted datetime'] #.dt.strftime('%Y-%m-%d %H:%M')
  df.drop(['adjusted datetime'], axis=1, inplace=True)

In [13]:
def season_creator (df,start_date,end_date,season_ranges):
    # Convert the date column to datetime
    df['date2'] = pd.to_datetime(df['date'])

    # Initialize the 'season' column with default value 0
    df['season'] = 0

    # Assign the season codes based on the date ranges
    for start_date, end_date, season_code in season_ranges:
        start = pd.to_datetime(start_date)
        end = pd.to_datetime(end_date)
        df.loc[(df['date2'] >= start) & (df['date2'] <= end), 'season'] = season_code
    df.drop(['date2'], axis=1, inplace=True)


# Bratteberg

In [609]:
school_name = 'bratteberg'
class1 = 141
class2 = 142
class3 = 210
class4 = 'Øst brakker'
occupants_replace =  { 'Øst brakker': 25 , 141:28 , 142:14,  210: 18 }  # for bratteberg


In [30]:
values_temp2   =  ['swc_f', 'swc_r', 'temp_v', 'temp_f', 'temp_s', 'press_a', 'temp_a', 'rh_a','enth_a','temp_o', 'rh_o','pmv', 'ppd']
values_air2    =  ['temp_a', 'rh_a', 'co2_a', 'voc_a', 'bright_a', 'rdn_a', 'enth_a','pmv', 'ppd', 'temp_o', 'rh_o', 'winds_o']
values_health2 =  ['temp_f','temp_a', 'rh_a', 'co2_a', 'voc_a', 'bright_a','hura_a','pm1_a', 'pm2.5_a','temp_o', 'rh_o','winds_o']
values_bright2 =  ['temp_a', 'bright_a', 'temp_o', 'rh_o','winds_o', 'rain_o', 'enth_o']
values_noise2  =  ['swc_f', 'swc_r', 'temp_v', 'temp_f','co2_a', 'voc_a', 'sound_a']

In [610]:
folder_year    = '2023'


start_date = '2023-03-06'
end_date  =  '2023-04-25'

In [15]:
folder_year    = '2024'

start_date = '2024-02-26'
end_date  =  '2024-03-08'

In [57]:
folder_year    = 'all years'

start_date = '2023-03-06'
end_date  =  '2024-03-08'

## Klasserom 141

In [611]:
class_dir    = '141'
school_name  = 'bratteberg'
# start_hour = 6
# end_hour   = 11

another_interpolate_col = airthingsPlus + weather_volda
sd_fill_col =  ['swc_f', 'swc_r', 'temp_s'] # [] #
sd_meaninterpolate_col = ['temp_v','temp_f']   #   []    #

interpolate_col = sd_fill_col+ sd_meaninterpolate_col + another_interpolate_col


## Klasserom 142

In [106]:
class_dir = '142'
school_name = 'bratteberg'
# start_hour = 6
# end_hour   = 11

another_interpolate_col = airthingsPlus + weather_volda
sd_fill_col =  ['swc_f', 'swc_r', 'temp_s'] # [] #
sd_meaninterpolate_col = ['temp_v','temp_f']   #   []    #

interpolate_col = sd_fill_col+ sd_meaninterpolate_col + another_interpolate_col


## Klasserom 210

In [196]:
class_dir   = '210'
school_name = 'bratteberg'
# start_hour = 6
# end_hour   = 11

another_interpolate_col = airthingsPlus + weather_volda

sd_fill_col            = ['swc_r', 'temp_s'] # 
sd_meaninterpolate_col = ['temp_v']          #   

interpolate_col = sd_fill_col  + sd_meaninterpolate_col + another_interpolate_col

## Klasserom Øst Brakker

In [241]:
class_dir   = 'Øst brakker'
school_name = 'bratteberg'
# start_hour = 6
# end_hour   = 11

another_interpolate_col = airthingsPro + weather_volda

sd_fill_col             = [] #[ 'swc_r']
sd_meaninterpolate_col  = [] #['temp_v']

interpolate_col = sd_fill_col+ sd_meaninterpolate_col + another_interpolate_col

# Øyra

In [629]:
class1 = 302
class2 = 303
class3 = 339
occupants_replace =  { 302:25   , 303:26    , 339:27 } # for øyra

sd_fill_col            =   [ 'temp_s']
sd_meaninterpolate_col =   ['temp_v','vent_v']   #    

interpolate_col =  sd_fill_col+ sd_meaninterpolate_col + another_interpolate_col

In [630]:
folder_year = '2023'
start_date  = '2023-03-13'
end_date    = '2023-04-18'

#  class result from 302, and 303

In [ ]:
folder_year = '2024'
start_date  = '2024-04-03'
end_date    = '2024-04-17'

#  class result from 303

In [263]:
folder_year = 'all years'
start_date  = '2023-03-13'
end_date    = '2024-04-18'

#  class result from 303

## Klasserom 302

In [631]:
class_dir = '302'
school_name = 'øyra'
# start_hour = 6
# end_hour   = 12

another_interpolate_col = airthingsPro + weather_volda
interpolate_col = sd_fill_col+ sd_meaninterpolate_col + another_interpolate_col

## Klasserom 303

In [437]:
class_dir = '303'
school_name = 'øyra'
# start_hour = 6
# end_hour   = 12

another_interpolate_col = airthingsPro + weather_volda
interpolate_col = sd_fill_col+ sd_meaninterpolate_col + another_interpolate_col

## Klasserom 339

In [ ]:
class_dir = '339'
school_name = 'øyra'
# start_hour = 6
# end_hour   = 12

another_interpolate_col = airthingsPro + weather_volda
interpolate_col = sd_fill_col+ sd_meaninterpolate_col + another_interpolate_col

# Brannfjell

In [65]:
season_ranges = [
    ('2023-03-03', '2023-04-25', 1),
    ('2023-11-10', '2023-11-30', 2),
    ('2024-02-02', '2024-03-10', 3),
    ('2024-04-07', '2024-04-30', 4)]

In [66]:
class1 = 11
class2 = 23
class3 = 27
class4 = 31
occupants_replace =  { 11:30   , 23:26    , 27:31 } # for brannfjell

In [67]:
# Batch 1
# REMEMBER :  class1 : 23, 27

folder_year    = '2023'

sd_fill_col =   [ 'temp_s']  # []     #
sd_meaninterpolate_col =  ['temp_v', 'co2_v']  #     []   #
another_interpolate_col = airthingsPlus + weather_oslo
interpolate_col = sd_fill_col+ sd_meaninterpolate_col + another_interpolate_col

start_date = '2023-03-22'
end_date  =  '2023-04-23'

In [68]:
# Batch 2
# REMEMBER :  class result : 11, 23, 27

folder_year    = '2023'

sd_fill_col    = []     #  [ 'temp_s']  # []     #
sd_meaninterpolate_col =   ['temp_v', 'co2_v']  #     []   #
another_interpolate_col = airthingsPlus + weather_oslo
interpolate_col = sd_fill_col+ sd_meaninterpolate_col + another_interpolate_col

start_date = '2023-11-13'
end_date  =  '2023-11-24'

In [69]:
# Batch 3
# REMEMBER : class result : 23, 27

folder_year    = '2024'

sd_fill_col    = []     #  [ 'temp_s']  # []     #
sd_meaninterpolate_col =    ['temp_v', 'co2_v']  #     []   #
another_interpolate_col = airthingsPlus + weather_oslo
interpolate_col = sd_fill_col+ sd_meaninterpolate_col + another_interpolate_col

start_date = '2024-02-26'
end_date  =  '2024-03-08'

In [70]:
# Batch 4
# REMEMBER :  class result : 11, 23, 27
folder_year    = '2024'
sd_fill_col    = []     #  [ 'temp_s']  # []     #
sd_meaninterpolate_col =  ['temp_v', 'co2_v']  #     []   #
another_interpolate_col = airthingsPlus + weather_oslo
interpolate_col = sd_fill_col+ sd_meaninterpolate_col + another_interpolate_col

start_date = '2024-04-15'
end_date  =  '2024-04-30'

In [71]:
# Batch 4
# REMEMBER :  class result : 11, 23, 27
folder_year    = 'all years'
sd_fill_col    = []     #  [ 'temp_s']  # []     #
sd_meaninterpolate_col =  ['temp_v', 'co2_v']  #     []   #
another_interpolate_col = airthingsPlus + weather_oslo
interpolate_col = sd_fill_col+ sd_meaninterpolate_col + another_interpolate_col

start_date = '2023-03-03'
end_date  =  '2024-04-30'

## Klasserom 11

In [72]:
class_dir = '11'
school_name = 'brannfjell'

## Klasserom 23

In [73]:
class_dir = '23'
school_name = 'brannfjell'

## Klasserom 27

In [74]:
class_dir = '27'
school_name = 'brannfjell'

## Klasserom 31

In [75]:
class_dir = '31'
school_name = 'brannfjell'

# KubenVGS

In [ ]:
class1 = 1707
class2 = 1713
class3 = 2707
class4 = 2713
class5 = 3707
class6 = 3708
class7 = 3713
occupants_replace =  { class1:30   , class2:18    , class3:30 , class4:17   ,class5: 20   ,class6: 18   ,class7:  25       }
airthingsDevice = airthingsPro
school_name     = 'kubenVGS'
end_file_name  = 'classes'
weather_data    = weather_oslo + weather_extra
another_interpolate_col = airthingsDevice + weather_data

## Klasserom 1707

In [ ]:
class_dir = str(class1)
school_name     = 'kubenVGS'
# start_date = '2023-03-16'
# end_date  =  '2023-11-25'
sd_fill_col = [] #[ 'temp_s']
sd_meaninterpolate_col =  []  #['temp_v', 'co2_v']
another_interpolate_col = airthingsDevice + weather_data
interpolate_col =  sd_fill_col+ another_interpolate_col #+ sd_meaninterpolate_col 

## Klasserom 1713

In [ ]:
class_dir = str(class2)
school_name     = 'kubenVGS'
# start_date = '2023-03-16'
# end_date  =  '2023-11-25'
sd_fill_col = [] #[ 'temp_s']
sd_meaninterpolate_col =  []  #['temp_v', 'co2_v']
another_interpolate_col = airthingsDevice + weather_data
interpolate_col =  sd_fill_col+ another_interpolate_col #+ sd_meaninterpolate_col 

## Klasserom 2707

In [ ]:
class_dir = str(class3)
school_name     = 'kubenVGS'
# start_date = '2023-03-16'
# end_date  =  '2023-11-25'
sd_fill_col = [] #[ 'temp_s']
sd_meaninterpolate_col =  []  #['temp_v', 'co2_v']
another_interpolate_col = airthingsDevice + weather_data
interpolate_col =  sd_fill_col+ another_interpolate_col #+ sd_meaninterpolate_col 

## Klasserom 2713

In [ ]:
class_dir = str(class4)
school_name     = 'kubenVGS'
# start_date = '2023-03-16'
# end_date  =  '2023-11-25'
sd_fill_col = [] #[ 'temp_s']
sd_meaninterpolate_col =  []  #['temp_v', 'co2_v']
another_interpolate_col = airthingsDevice + weather_data
interpolate_col =  sd_fill_col+ another_interpolate_col #+ sd_meaninterpolate_col 

## Klasserom 3707

In [ ]:
class_dir = str(class5)
school_name     = 'kubenVGS'
# start_date = '2023-03-16'
# end_date  =  '2023-11-25'
sd_fill_col = [] #[ 'temp_s']
sd_meaninterpolate_col =  []  #['temp_v', 'co2_v']
another_interpolate_col = airthingsDevice + weather_data
interpolate_col =  sd_fill_col+ another_interpolate_col #+ sd_meaninterpolate_col 

## Klasserom 3708

In [ ]:
class_dir = str(class6)
school_name     = 'kubenVGS'
# start_date = '2023-03-16'
# end_date  =  '2023-11-25'
sd_fill_col = [] #[ 'temp_s']
sd_meaninterpolate_col =  []  #['temp_v', 'co2_v']
another_interpolate_col = airthingsDevice + weather_data
interpolate_col =  sd_fill_col+ another_interpolate_col #+ sd_meaninterpolate_col 

## Klasserom 3713

In [ ]:
class_dir = str(class7)
school_name     = 'kubenVGS'
# start_date = '2023-03-16'
# end_date  =  '2023-11-25'
sd_fill_col = [] #[ 'temp_s']
sd_meaninterpolate_col =  []  #['temp_v', 'co2_v']
another_interpolate_col = airthingsDevice + weather_data
interpolate_col =  sd_fill_col+ another_interpolate_col #+ sd_meaninterpolate_col 

# Processor #1

## SD Anlegg

In [555]:
manuf_id    = 'sdanlegg'
input_file_directory    = platform_directory + '/' + folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'+class_dir+'/'

In [556]:
folder_path1 =  input_file_directory # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('.csv')] # get list of csv files

dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_csv(file_path, delimiter = ',') #.iloc[3:] # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list
    
# concatenate dataframes in the list into a single dataframe
sdanlegg = pd.concat(dfs1, ignore_index=True) #.fillna(method='backfill')

In [ ]:
sdanlegg

In [ ]:
datetime_indexer(sdanlegg )
sdanlegg = sdanlegg[sdanlegg['date'].between(start_date,end_date) 
                    # & sdanlegg['time'].between(start_time,end_time)
                    ]
sdanlegg = sdanlegg.iloc[:,7: ]
sdanlegg = sdanlegg.resample(time_series_resample).quantile(0.5)
#sdanlegg = sdanlegg.resample(time_series_resample).mean()
sdanlegg = sdanlegg.resample(desample_option).fillna(method='bfill').round(2)

#select special date
sdanlegg = sdanlegg.reset_index()
sdanlegg['date'] = pd.to_datetime(sdanlegg['datetime']).dt.strftime('%Y-%m-%d')
sdanlegg  = sdanlegg[sdanlegg['date'].between(start_date,end_date)].drop('date', axis=1).set_index('datetime')

In [ ]:
sdanlegg

## Airthings

In [560]:
manuf_id    = 'airthings'
input_file_directory    = platform_directory + '/' + folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'+class_dir+'/'

In [561]:
folder_path1 = input_file_directory # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('.csv')] # get list of csv files

dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_csv(file_path, delimiter = ',').iloc[3:] # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list

In [ ]:
# concatenate dataframes in the list into a single dataframe
airthings = pd.concat(dfs1, ignore_index=True) #.fillna(method='backfill')
datetime_indexer(airthings )
airthings = airthings[airthings['date'].between(start_date,end_date) 
                    #   & airthings['time'].between(start_time,end_time)
                      ]
airthings = airthings.iloc[:,7: ]
airthings = airthings.resample(time_series_resample).quantile(0.5)
#airthings = airthings.resample(time_series_resample).mean()
airthings = airthings.resample(desample_option).fillna(method='bfill').round(2)

#select special date
airthings = airthings.reset_index()
airthings['date'] = pd.to_datetime(airthings['datetime']).dt.strftime('%Y-%m-%d')
airthings  = airthings[airthings['date'].between(start_date,end_date)].drop('date', axis=1).set_index('datetime')

In [ ]:
# Initialize psychrometric library
psychrolib.SetUnitSystem(psychrolib.SI)  # Set units to SI (Celsius, Pa, kg/kg, etc.)

# Calculate enthalpy using dry bulb temperature and relative humidity
enthalpy = []
humid_rat = []
wet_bulb = []
for index, row in airthings.iterrows():
    tdb = row['temp_a']  # Dry bulb temperature in Celsius
    rh = row['rh_a']/100  # Relative humidity (fraction, not percentage)
    pa = row['press_a']*100  #atmospheric pressure in Pascal

    h = psychrolib.GetHumRatioFromRelHum(tdb, rh, pa)  # Calculate humidity ratio
    wb = psychrolib.GetTWetBulbFromRelHum(tdb, rh, pa )
    #append all calculation
    enthalpy.append(psychrolib.GetMoistAirEnthalpy(tdb, h) / 1000)  # Calculate enthalpy (in kJ/kg) using tdb and humid ratio, Convert J/kg to kJ/kg
    humid_rat.append(h)
    wet_bulb.append(wb)

In [564]:
# Add enthalpy column to the DataFrame
airthings['enth_a']   = np.around(enthalpy,2)
airthings['hura_a']   = humid_rat
airthings['hura_a']   = np.around((airthings['hura_a']  *1000),2)

In [565]:
pmv_ppd_generator(airthings)

In [ ]:
airthings

## IS Data

In [567]:
manuf_id    = 'niluapp'
input_file_directory      = platform_directory + '/' + folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/'+ manuf_id + '/' +class_dir+'/'

In [ ]:
input_file_directory

In [569]:
folder_path1 =  input_file_directory # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('.csv')] # get list of csv files

dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_csv(file_path, delimiter = ',') #.iloc[3:] # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list
    
# concatenate dataframes in the list into a single dataframe
niluapp = pd.concat(dfs1, ignore_index=True) #.fillna(method='backfill')

datetime_indexer(niluapp )
#niluapp = niluapp[niluapp['date'].between(start_date,end_date) & niluapp['time'].between(start_time,end_time)]
niluapp = niluapp.iloc[:,8: ]
niluapp[response_counting] = 1
niluapp.insert(0, response_counting, niluapp.pop(response_counting))


#select special date
niluapp = niluapp.reset_index()
niluapp['date'] = pd.to_datetime(niluapp['datetime']).dt.strftime('%Y-%m-%d')
niluapp  = niluapp[niluapp['date'].between(start_date,end_date)].drop('date', axis=1).set_index('datetime')

quantitative_niluapp(niluapp)

In [570]:
answer_replace        = { 0:np.nan, 1:2 , 2:1 ,     3:-1 , 4:-2 }
#answer_replace        = { 0:np.nan , 1:-2 , 2:-1 , 3:1 , 4:2 }
#answer_replace        = { 0:np.nan , 1:1 , 2:1 , 3:-1 , 4:-1 }
answer_replace_coldhot = {1:3 , 2:2 , 3:1 , 4:0 , 5:-1 , 6:-2 , 7:-3 }
#answer_replace        = {0:0 ,1:1 , 2:0.5, 3:-0.5, 4:-1 }
niluapp['temp_coldhot'] = niluapp['temp_coldhot'].replace(answer_replace_coldhot)
niluapp.iloc[:,1:6] = niluapp.iloc[:,1:6].replace(answer_replace)   #replace the codification of response's answers

In [571]:
niluapp1 = niluapp[is_col_multi].resample(survey_resample).quantile(0.5).round(0)      #.median()
niluapp2 = niluapp.drop(is_col_multi, axis=1).resample(survey_resample).sum()
niluapp  = niluapp1 .join(niluapp2, how='outer')                       #resampling and summarize mainquestion's value
niluapp  = niluapp[~(niluapp.iloc[:,6:21] == 0).all(axis=1)]         #delete a rows if they have 0 values in main questionscolumns

## outdoor

In [572]:
manuf_id    = 'seklima'
input_file_directory  = platform_directory + '/' + folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'

In [573]:
folder_path1 = input_file_directory # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('.csv')] # get list of csv files

dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_csv(file_path).iloc[3:] # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list
    
# concatenate dataframes in the list into a single dataframe
outdoor = pd.concat(dfs1, ignore_index=True) #.fillna(method='backfill')


datetime_indexer(outdoor )
#outdoor = outdoor[outdoor['date'].between(start_date,end_date) & outdoor['time'].between(start_time,end_time)]
outdoor = outdoor.iloc[:,6: ]
outdoor = outdoor.resample(time_series_resample).mean().interpolate(method='index')
#outdoor = outdoor.resample(desample_option).bfill().round(2).dropna(how='all',axis=0)

#select special date
outdoor = outdoor.reset_index()
outdoor['date'] = pd.to_datetime(outdoor['datetime']).dt.strftime('%Y-%m-%d')
outdoor  = outdoor[outdoor['date'].between(start_date,end_date)].drop('date', axis=1).set_index('datetime')



# datetime_indexer(outdoor )
# outdoor = outdoor[outdoor['date'].between(start_date,end_date) 
#                 #   & outdoor['time'].between(start_time,end_time)
#                   ]
# outdoor2 = outdoor.iloc[:,6: ].drop(columns=['rain_o'], axis=1)
# outdoor2 = outdoor2.resample(time_series_resample).mean().interpolate(method='index').round(3)
# outdoor2['rain_o'] = outdoor['rain_o'].resample(time_series_resample).sum().interpolate(method='index')
# outdoor = outdoor.resample(desample_option).bfill().round(2).dropna(how='all',axis=0)

# #select special date
# outdoor2 = outdoor2.reset_index()
# outdoor2['date'] = pd.to_datetime(outdoor2['datetime']).dt.strftime('%Y-%m-%d')
# outdoor  = outdoor2[outdoor2['date'].between(start_date,end_date)].drop('date', axis=1).set_index('datetime')

In [574]:
# Initialize psychrometric library
psychrolib.SetUnitSystem(psychrolib.SI)  # Set units to SI (Celsius, Pa, kg/kg, etc.)

# Calculate enthalpy using dry bulb temperature and relative humidity
enthalpy = []
humid_rat = []
wet_bulb = []
for index, row in outdoor.iterrows():
    tdb = row['temp_o']  # Dry bulb temperature in Celsius
    rh = row['rh_o']/100  # Relative humidity (fraction, not percentage)
    pa = row['press_o']*100 #atmospheric pressure in Pascal

    h = psychrolib.GetHumRatioFromRelHum(tdb, rh, pa)  # Calculate humidity ratio
    wb = psychrolib.GetTWetBulbFromRelHum(tdb, rh, pa )
    #append all calculation
    enthalpy.append(psychrolib.GetMoistAirEnthalpy(tdb, h) / 1000)  # Calculate enthalpy (in kJ/kg) using tdb and humid ratio, Convert J/kg to kJ/kg
    humid_rat.append(h)
    wet_bulb.append(wb)

In [575]:
# Add enthalpy column to the DataFrame
outdoor['enth_o']   = np.around(enthalpy,2)
outdoor['hura_o'] = humid_rat
outdoor['hura_o']   = np.around((outdoor['hura_o']  *1000),2)

## Concatenate

In [ ]:
combined1 =sdanlegg .join(airthings, how='outer').join(outdoor, how='outer').join(niluapp, how='outer')
# combined1 =airthings.join(outdoor, how='outer').join(niluapp, how='outer')

#numeric correction (Interpolating & Filling)
combined1.loc[:,interpolate_col] = combined1.loc[:,interpolate_col].interpolate(method= 'index').round(2)
combined1.loc[:,sd_fill_col] = combined1.loc[:,sd_fill_col].fillna(method='ffill')
combined1['hura_d'] = combined1['hura_a'] - combined1['hura_o']
#selector
datedayhour_creator(combined1)
combined1 = combined1[(combined1['date'].between(start_date,end_date)) 
                    #  &(combined1['day'].between(start_day,end_day)) &
                    #  &(combined1['hour'].between(start_hour,end_hour))
                     ] .drop(['date', 'day'],axis = 1).set_index(['datetime'])#.dropna(how= 'any')
combined1 [room_col_name]   = class_dir
combined1 [school_col_name] = school_name

## Export

In [577]:
#put spesific Directory and File Name.
directory_path = platform_directory + '/' + folder_project + '/' + folder_ready + '/'+ folder_year + '/' + school_name + '/' + folder_export + '/'
combined1.reset_index(inplace=True)

combined1['date'] = combined1['datetime'].dt.strftime('%Y-%m-%d')
combined1['time'] = combined1['datetime'].dt.strftime('%H:%M')
filename   = school_name+'_'+ class_dir + '_combined all.csv'
filenamexl = school_name+'_'+ class_dir + '_combined all.xlsx'

# combined1 = combined1.drop(['datetime'], axis = 1)
# save dataframe to CSV file in specified directory
directory_export = directory_path + data_for1 + '/' + time_series_resample + '_'
# combined1.to_excel( directory_export + filenamexl, index=False)
combined1.to_csv( directory_export + filename, index=False)

# Processor #2

## SD Anlegg

In [578]:
manuf_id    = 'sdanlegg'
input_file_directory      = platform_directory + '/' + folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/' +class_dir+'/'

In [579]:
folder_path1 =  input_file_directory # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('.csv')] # get list of csv files

dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_csv(file_path, delimiter = ',') #.iloc[3:] # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list

In [580]:
# concatenate dataframes in the list into a single dataframe
sdanlegg = pd.concat(dfs1, ignore_index=True) #.fillna(method='backfill')
datetime_indexer(sdanlegg )
sdanlegg1 = sdanlegg.loc[:,sd_fill_col].resample(survey_resample).quantile(0.5) #.round(2)
sdanlegg1 = sdanlegg1.resample(minutely_resample).bfill().round(2)
sdanlegg2 = sdanlegg.loc[:,sd_meaninterpolate_col].resample(survey_resample).quantile(0.75) #.round(2)
#sdanlegg2 = sdanlegg.loc[:,sd_meaninterpolate_col].resample(time_series_resample).mean() #.round(2)
sdanlegg2 = sdanlegg2.resample(minutely_resample).bfill().round(2) #.asfreq()
sdanlegg = sdanlegg1.join(sdanlegg2, how = 'outer')

#select special date
sdanlegg = sdanlegg.reset_index()
sdanlegg['date'] = pd.to_datetime(sdanlegg['datetime']).dt.strftime('%Y-%m-%d')
sdanlegg  = sdanlegg[sdanlegg['date'].between(start_date,end_date)].drop('date', axis=1).set_index('datetime')

## Airthings

In [581]:
manuf_id    = 'airthings'
input_file_directory      = platform_directory + '/' + folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/' +class_dir+'/'

In [582]:
folder_path1 = input_file_directory # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('.csv')] # get list of csv files

dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_csv(file_path, delimiter = ',').iloc[3:] # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list

In [583]:
# concatenate dataframes in the list into a single dataframe
airthings = pd.concat(dfs1, ignore_index=True) #.fillna(method='backfill')
datetime_indexer(airthings )

In [584]:
# concatenate dataframes in the list into a single dataframe
airthings = pd.concat(dfs1, ignore_index=True) #.fillna(method='backfill')
datetime_indexer(airthings )
airthings = airthings.iloc[:,7: ]
airthings = airthings.resample(survey_resample).quantile(0.5)
#airthings = airthings.resample(time_series_resample).mean()
airthings = airthings.resample(minutely_resample).bfill().round(2)

#select special date
airthings = airthings.reset_index()
airthings['date'] = pd.to_datetime(airthings['datetime']).dt.strftime('%Y-%m-%d')
airthings  = airthings[airthings['date'].between(start_date,end_date)].drop('date', axis=1).set_index('datetime')

In [ ]:
# Initialize psychrometric library
psychrolib.SetUnitSystem(psychrolib.SI)  # Set units to SI (Celsius, Pa, kg/kg, etc.)

# Calculate enthalpy using dry bulb temperature and relative humidity
enthalpy = []
humid_rat = []
wet_bulb = []
for index, row in airthings.iterrows():
    tdb = row['temp_a']  # Dry bulb temperature in Celsius
    rh = row['rh_a']/100  # Relative humidity (fraction, not percentage)
    pa = row['press_a']*100  #atmospheric pressure in Pascal

    h = psychrolib.GetHumRatioFromRelHum(tdb, rh, pa)  # Calculate humidity ratio
    wb = psychrolib.GetTWetBulbFromRelHum(tdb, rh, pa )
    #append all calculation
    enthalpy.append(psychrolib.GetMoistAirEnthalpy(tdb, h) / 1000)  # Calculate enthalpy (in kJ/kg) using tdb and humid ratio, Convert J/kg to kJ/kg
    humid_rat.append(h)
    wet_bulb.append(wb)

In [586]:
# Add enthalpy column to the DataFrame
airthings['enth_a']   = np.around(enthalpy,2)
airthings['hura_a']   = humid_rat
airthings['hura_a']   = np.around((airthings['hura_a']  *1000),2)

In [587]:
pmv_ppd_generator(airthings)

## IS Data

In [588]:
manuf_id    = 'niluapp'
input_file_directory      = platform_directory + '/' + folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/'+ manuf_id + '/' +class_dir+'/'
folder_path1 =  input_file_directory # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('.csv')] # get list of csv files

dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_csv(file_path, delimiter = ',') #.iloc[3:] # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list
    
# concatenate dataframes in the list into a single dataframe
niluapp = pd.concat(dfs1, ignore_index=True) #.fillna(method='backfill')


datetime_indexer(niluapp )
#niluapp = niluapp[niluapp['date'].between(start_date,end_date) & niluapp['time'].between(start_time,end_time)]
niluapp = niluapp.iloc[:,8: ]

#select special date
niluapp = niluapp.reset_index()
niluapp['date'] = pd.to_datetime(niluapp['datetime']).dt.strftime('%Y-%m-%d')
niluapp  = niluapp[niluapp['date'].between(start_date,end_date)].drop('date', axis=1).set_index('datetime')

quantitative_niluapp(niluapp)

In [589]:
answer_replace        = { 0:np.nan, 1:2 , 2:1 ,     3:-1 , 4:-2 }
#answer_replace        = { 0:np.nan , 1:-2 , 2:-1 , 3:1 , 4:2 }
#answer_replace        = { 0:np.nan , 1:1 , 2:1 , 3:-1 , 4:-1 }
answer_replace_coldhot = {0:np.nan , 1:-3 , 2:-2 , 3:-1 , 4:0 , 5:1 , 6:2 , 7:3 }
#answer_replace        = {0:0 ,1:1 , 2:0.5, 3:-0.5, 4:-1 }
niluapp.iloc[:,:5] = niluapp.iloc[:,0:5].replace(answer_replace)   #replace the codification of response's answers
niluapp['temp_coldhot'] = niluapp['temp_coldhot'].replace(answer_replace_coldhot)
niluapp[response_counting] = 1

## outdoor

In [590]:
manuf_id    = 'seklima'
input_file_directory  = platform_directory + '/' + folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'

In [591]:
folder_path1 = input_file_directory # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('.csv')] # get list of csv files

dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_csv(file_path, delimiter = ',').iloc[3:] # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list

In [592]:
# concatenate dataframes in the list into a single dataframe
outdoor = pd.concat(dfs1, ignore_index=True) #.fillna(method='backfill')
datetime_indexer(outdoor )
outdoor = outdoor.iloc[:,6: ]
outdoor = outdoor.resample(time_series_resample).quantile(0.75)
#outdoor = outdoor.resample(survey_resample).interpolate(method='index').round(2)
outdoor = outdoor.resample(minutely_resample).bfill().round(2)
outdoor['rain_o'] = outdoor['rain_o'].resample(time_series_resample).sum().interpolate(method='index')

#select special date
outdoor = outdoor.reset_index()
outdoor['date'] = pd.to_datetime(outdoor['datetime']).dt.strftime('%Y-%m-%d')
outdoor  = outdoor[outdoor['date'].between(start_date,end_date)].drop('date', axis=1).set_index('datetime')

In [ ]:
# Initialize psychrometric library
psychrolib.SetUnitSystem(psychrolib.SI)  # Set units to SI (Celsius, Pa, kg/kg, etc.)

# Calculate enthalpy using dry bulb temperature and relative humidity
enthalpy = []
humid_rat = []
wet_bulb = []
for index, row in outdoor.iterrows():
    tdb = row['temp_o']  # Dry bulb temperature in Celsius
    rh = row['rh_o']/100  # Relative humidity (fraction, not percentage)
    pa = row['press_o']*100  #atmospheric pressure in Pascal

    h = psychrolib.GetHumRatioFromRelHum(tdb, rh, pa)  # Calculate humidity ratio
    wb = psychrolib.GetTWetBulbFromRelHum(tdb, rh, pa )
    #append all calculation
    enthalpy.append(psychrolib.GetMoistAirEnthalpy(tdb, h) / 1000)  # Calculate enthalpy (in kJ/kg) using tdb and humid ratio, Convert J/kg to kJ/kg
    humid_rat.append(h)
    wet_bulb.append(wb)

In [594]:
# Add enthalpy column to the DataFrame
outdoor['enth_o']   = np.around(enthalpy,2)
outdoor['hura_o']   = humid_rat
outdoor['hura_o']   = np.around((outdoor['hura_o']  *1000),2)

## Concatenate

In [ ]:
# combined2 = airthings.join(outdoor, how='outer').join(niluapp, how='outer')
combined2 = sdanlegg.join(airthings, how='outer').join(outdoor, how='outer').join(niluapp, how='outer')

combined2.loc[:,interpolate_col] = combined2.loc[:,interpolate_col].fillna(method='ffill').round(2)
combined2['hura_d'] = combined2['hura_a'] + combined2['hura_o']
#selector
datedayhour_creator(combined2)
combined2 = combined2[(combined2['date'].between(start_date,end_date) & combined2['day'].between(start_day,end_day) & combined2['hour'].between(start_hour,end_hour))].set_index(['datetime']) .drop(['date', 'day'],axis = 1).dropna(subset=is_col_main)
combined2[room_col_name]= class_dir
combined2[school_col_name] = school_name

## Export

In [596]:
#put spesific Directory and File Name.
directory_path = platform_directory + '/' + folder_project + '/' + folder_ready+ '/' + folder_year + '/' + school_name + '/' + folder_export + '/'

combined2.reset_index(inplace=True)
combined2['date'] = combined2['datetime'].dt.strftime('%Y-%m-%d')
combined2['time'] = combined2['datetime'].dt.strftime('%H:%M')
filename = school_name+'_'+class_dir + '_combined all.csv'
filenamexl = school_name+'_'+class_dir + '_combined all.xlsx'

directory_export = directory_path + data_for2+ '/' + survey_resample + '_'
combined2.to_excel( directory_export + filenamexl, index=False)
combined2.to_csv( directory_export + filename, index=False)

In [ ]:
print(directory_export+filenamexl)

In [ ]:
print(do you know Stop?)

# Processor #3

## SD Anlegg

In [ ]:
another_interpolate_col = [ co2_name,temp_data_name, rel_hum_name, press_name, voc_name,bright_name, radon_name ]
sd_meaninterpolate_col = ['temp_v', 'co2_v']

interpolate_col =  sd_meaninterpolate_col + another_interpolate_col

In [ ]:
manuf_id    = 'sdanlegg'
input_file_directory    = platform_directory + '/' + folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'+class_dir+'/'

In [ ]:
folder_path1 =  input_file_directory # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('.csv')] # get list of csv files

dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_csv(file_path, delimiter = ',') #.iloc[3:] # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list
    
# concatenate dataframes in the list into a single dataframe
sdanlegg = pd.concat(dfs1, ignore_index=True) #.fillna(method='backfill')

datetime_indexer(sdanlegg )
#sdanlegg = sdanlegg[sdanlegg['date'].between(start_date,end_date) & sdanlegg['time'].between(start_time,end_time)]
sdanlegg = sdanlegg.iloc[:,7: ]
sdanlegg = sdanlegg.resample(time_series_resample).quantile(0.5)
#sdanlegg = sdanlegg.resample(time_series_resample).mean()
sdanlegg = sdanlegg.resample(desample_option).fillna(method='bfill').round(2)

#select special date
sdanlegg = sdanlegg.reset_index()
sdanlegg['date'] = pd.to_datetime(sdanlegg['datetime']).dt.strftime('%Y-%m-%d')
sdanlegg = sdanlegg.set_index(['datetime'])[sd_meaninterpolate_col]
# sdanlegg  = sdanlegg[sdanlegg['date'].between(start_date,end_date)].drop('date', axis=1).set_index('datetime')

## Airthings

In [ ]:
manuf_id    = 'airthings'
input_file_directory    = platform_directory + '/' + folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/' + manuf_id + '/'+class_dir+'/'
airthingsDevice = []

In [ ]:
folder_path1 = input_file_directory # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('.csv')] # get list of csv files

dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_csv(file_path, delimiter = ',').iloc[3:] # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list

In [ ]:
# concatenate dataframes in the list into a single dataframe
airthings = pd.concat(dfs1, ignore_index=True) #.fillna(method='backfill')
datetime_indexer(airthings )
#airthings = airthings[airthings['date'].between(start_date,end_date) & airthings['time'].between(start_time,end_time)]
airthings = airthings.iloc[:,7: ]
airthings = airthings.resample(time_series_resample).quantile(0.5)
#airthings = airthings.resample(time_series_resample).mean()
airthings = airthings.resample(desample_option).fillna(method='bfill').round(2)

#select special date
airthings = airthings.reset_index()
airthings['date'] = pd.to_datetime(airthings['datetime']).dt.strftime('%Y-%m-%d')
airthings = airthings.set_index(['datetime'])
# airthings  = airthings[airthings['date'].between(start_date,end_date)].drop('date', axis=1).set_index('datetime')

## IS Data

In [ ]:
manuf_id    = 'niluapp'
input_file_directory      = platform_directory + '/' + folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/'+ manuf_id + '/' +class_dir+'/'
input_file_directory
folder_path1 =  input_file_directory # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('.csv')] # get list of csv files

dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_csv(file_path, delimiter = ',') #.iloc[3:] # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list
    
# concatenate dataframes in the list into a single dataframe
niluapp = pd.concat(dfs1, ignore_index=True) #.fillna(method='backfill')

datetime_indexer(niluapp )
#niluapp = niluapp[niluapp['date'].between(start_date,end_date) & niluapp['time'].between(start_time,end_time)]
niluapp = niluapp.iloc[:,8: ]
niluapp[response_counting] = 1
niluapp.insert(0, response_counting, niluapp.pop(response_counting))


#select special date
niluapp = niluapp.reset_index()
niluapp['date'] = pd.to_datetime(niluapp['datetime']).dt.strftime('%Y-%m-%d')


In [ ]:
niluapp['date'].unique()

## Concatenate

In [ ]:
# combined3 =sdanlegg .join(airthings, how='inner', on='datetime')
# combined1 =airthings.join(outdoor, how='outer').join(niluapp, how='outer')
combined3 = pd.concat([airthings,sdanlegg], axis= 'columns', ignore_index=False)#.drop('temp_s', axis=1)  #dropna(subset=['co2_v', 'co2_a' ], axis=0)
#numeric correction (Interpolating & Filling)
# combined3.loc[:,interpolate_col] = combined3.loc[:,interpolate_col].interpolate(method= 'index').round(2)
# combined3.loc[:,sd_fill_col] = combined3.loc[:,sd_fill_col].fillna(method='ffill')

#selector
datedayhour_creator(combined3)
combined3 = combined3[           (combined3['date'].isin  (niluapp['date'].unique())) &
                     (combined3['day'].between(start_day,end_day)) & (combined3['hour'].between(6,16)
                     )] .drop([ 'day'],axis = 1).set_index(['datetime'])
combined3 [room_col_name]   = class_dir
combined3 [school_col_name] = school_name
combined3 = combined3[~(combined3['temp_a'].isna() & combined3['temp_v'].isna())]
combined3#.dropna(subset=['co2_v', 'co2_a' ])
#numeric correction (Interpolating & Filling)

## Export

In [ ]:
#put spesific Directory and File Name.
directory_path = platform_directory + '/' + folder_project + '/' + folder_ready + '/'+ folder_year + '/' + school_name + '/' + folder_export + '/'
combined3.reset_index(inplace=True)

combined3['date'] = combined3['datetime'].dt.strftime('%Y-%m-%d')
combined3['time'] = combined3['datetime'].dt.strftime('%H:%M')
name_date = datetime.strptime((combined3[date_name].iloc[1]),'%Y-%m-%d').strftime('%Y%m%d')
filename   = school_name+'_'+ class_dir+ '_' + name_date + '.csv'
filenamexl = school_name+'_'+ class_dir+ '_' + name_date + '.xlsx'

time_zone_converter(combined3, 'UTC', 'Europe/Oslo')
datetime_indexer_noindex(combined3)
datedayhour_creator_noindex(combined3)
# save dataframe to CSV file in specified directory
directory_export = directory_path + data_for3 + '/' + time_series_resample + '_'
combined3.to_excel( directory_export + filenamexl, index=False)
combined3.to_csv( directory_export + filename, index=False)

In [ ]:
print(directory_export + filenamexl)

In [ ]:
sns.scatterplot(combined3 , x='co2_v', y= 'co2_a')
# sns.scatterplot(combined3 , x='temp_v', y= 'temp_a')

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# x_axis = 'co2_v'
# y_axis = 'co2_a'
x_axis = 'temp_v'
y_axis = 'temp_a'
regress_data = combined3.copy().dropna(axis=0)
# regress_data = regress_data
xb=regress_data[[x_axis]]
yb=regress_data[[y_axis]]


In [ ]:
Lin_reg = LinearRegression()
Lin_reg.fit(xb, yb)

y_pred = Lin_reg.predict(xb)#*back_scaling

print(Lin_reg.intercept_)
print(Lin_reg.coef_.flatten().round(4))
print(r2_score(yb,y_pred))

# COMBINER

In [ ]:
print("school is " +school_name +  ' with class: ' +  class_dir+ ' with resampling: ' + time_series_resample)
directory_path = platform_directory + '/' + folder_project + '/' + folder_ready+ '/' + folder_year + '/' + school_name + '/' + folder_export + '/'


In [ ]:
print((directory_path + data_for1 + '/' +school_name +'/') )   # replace with the folder path)

## Calibrating Data

In [ ]:
folder_path3 = directory_path + data_for3 + '/'    # replace with the folder path
csv_files3 = [f for f in os.listdir(folder_path3) if f.endswith('.xlsx') and f.startswith(time_series_resample + '_' + school_name + '_' )] # get list of csv files
dfs3 = [] # list to store dataframes

for file in csv_files3:
    file_path = os.path.join(folder_path3, file) # get full path of csv file
    df = pd.read_excel(file_path) # read csv file into a dataframe
    dfs3.append(df) # append dataframe to list

# concatenate dataframes in the list into a single dataframe
combined3 = pd.concat(dfs3, ignore_index=True)

#we convert time zone of dataframe, from GMT to Local
time_zone_converter(combined3, 'UTC', 'Europe/Oslo')
datetime_indexer_noindex(combined3)
datedayhour_creator_noindex(combined3)


#put spesific Directory and File Name.
directory_path = platform_directory + '/' + folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/' + folder_export + '/'

combined3['date'] = combined3['datetime'].dt.strftime('%Y-%m-%d')
combined3['time'] = combined3['datetime'].dt.strftime('%H:%M')
name_date = datetime.strptime((combined3[date_name].iloc[1]),'%Y-%m-%d').strftime('%Y%m%d')
filename = school_name+ '_' + name_date + '_combined all classes.csv'
filenamexl = school_name+ '_' + name_date + '_combined all classes.xlsx'

# save dataframe to CSV file in specified directory
directory_export = directory_path + data_for3 + '/'+ time_series_resample
combined3.to_excel( directory_export + '_1_' + filenamexl, index=False)
combined3.to_csv( directory_export + '_1_' + filename, index=False)
combined3.describe().to_excel( directory_export + '_descstat_3_' + school_name + '_' + name_date + '.xlsx', index=False)


## panel data

In [661]:
folder_path1 = directory_path + data_for1 + '/'    # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('all.csv') and f.startswith(time_series_resample)] # get list of csv files
dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_csv(file_path) # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list

# concatenate dataframes in the list into a single dataframe
combined1 = pd.concat(dfs1, ignore_index=True)

#we convert time zone of dataframe, from GMT to Local
time_zone_converter(combined1, 'UTC', 'Europe/Oslo')
datetime_indexer_noindex(combined1)
datedayhour_creator_noindex(combined1)


In [663]:
combined1['students'] = combined1['room_id'].replace(occupants_replace)
combined1['response_rate'] = combined1[response_counting] / combined1['students']

In [664]:
weight_1 = 1
weight_2 = 1#0.5  #1
weight_3 = 1#0.5  #1
weight_4 = 1

combined1['ad_air']    = (combined1['feel_air4']*weight_4 + combined1['feel_air3']*weight_3)/combined1[response_counting]
combined1['ad_temp']   = (combined1['feel_temp4']*weight_4 + combined1['feel_temp3']*weight_3)/combined1[response_counting]
combined1['ad_health'] = (combined1['feel_health4']*weight_4 + combined1['feel_health3']*weight_3)/combined1[response_counting]
combined1['ad_bright'] = (combined1['feel_bright4']*weight_4 + combined1['feel_bright3']*weight_3)/combined1[response_counting]
combined1['ad_noise']  = (combined1['feel_noise4']*weight_4 + combined1['feel_noise3']*weight_3)/combined1[response_counting]
combined1['as_air']    = (combined1['feel_air1']*weight_1 + combined1['feel_air2']*weight_2)/combined1[response_counting]
combined1['as_temp']   = (combined1['feel_temp1']*weight_1 + combined1['feel_temp2']*weight_2)/combined1[response_counting]
combined1['as_health'] = (combined1['feel_health1']*weight_1 + combined1['feel_health2']*weight_2)/combined1[response_counting]
combined1['as_bright'] = (combined1['feel_bright1']*weight_1 + combined1['feel_bright2']*weight_2)/combined1[response_counting]
combined1['as_noise']  = (combined1['feel_noise1']*weight_1 + combined1['feel_noise2']*weight_2)/combined1[response_counting]
combined1['pps'] = 1-combined1['ppd']

combined1['rd_air']    = (combined1['feel_air4']*weight_4 + combined1['feel_air3']*weight_3)/combined1['students']
combined1['rd_temp']   = (combined1['feel_temp4']*weight_4 + combined1['feel_temp3']*weight_3)/combined1['students']
combined1['rd_health'] = (combined1['feel_health4']*weight_4 + combined1['feel_health3']*weight_3)/combined1['students']
combined1['rd_bright'] = (combined1['feel_bright4']*weight_4 + combined1['feel_bright3']*weight_3)/combined1['students']
combined1['rd_noise']  = (combined1['feel_noise4']*weight_4 + combined1['feel_noise3']*weight_3)/combined1['students']
combined1['rs_air']    = (combined1['feel_air1']*weight_1 + combined1['feel_air2']*weight_2)/combined1['students']
combined1['rs_temp']   = (combined1['feel_temp1']*weight_1 + combined1['feel_temp2']*weight_2)/combined1['students']
combined1['rs_health'] = (combined1['feel_health1']*weight_1 + combined1['feel_health2']*weight_2)/combined1['students']
combined1['rs_bright'] = (combined1['feel_bright1']*weight_1 + combined1['feel_bright2']*weight_2)/combined1['students']
combined1['rs_noise']  = (combined1['feel_noise1']*weight_1 + combined1['feel_noise2']*weight_2)/combined1['students']

In [665]:
#put spesific Directory and File Name.
directory_path = platform_directory + '/' + folder_project + '/' + folder_ready + '/' + folder_year + '/' + school_name + '/' + folder_export + '/'

combined1['date'] = combined1['datetime'].dt.strftime('%Y-%m-%d')
combined1['time'] = combined1['datetime'].dt.strftime('%H:%M')
name_date = datetime.strptime((combined1[date_name].iloc[1]),'%Y-%m-%d').strftime('%Y%m%d')
filename = school_name+ '_' + name_date + '_combined all classes.csv'
filenamexl = school_name+ '_' + name_date + '_combined all classes.xlsx'

# #IF ALL YEARS
# filename = school_name + '_alldate.csv'
# filenamexl = school_name + '_alldate.xlsx'

# save dataframe to CSV file in specified directory
directory_export = directory_path + data_for1 + '/'+ time_series_resample
combined1.to_excel( directory_export + '_1_' + filenamexl, index=False)
combined1.to_csv( directory_export + '_1_' + filename, index=False)
combined1.describe().to_excel( directory_export + '_descstat_1_' + school_name + '_' + name_date + '.xlsx', index=False)

In [ ]:
print(folder_path1)
print(directory_export + '_1_' + filenamexl)

In [ ]:
print(combined1['room_id'].unique())
print(combined1['school_id'].unique())
print(combined1['time'].unique())
print(combined1[response_counting].sum())


print(directory_export + '_1_' + filenamexl)

## cross sectional data

In [ ]:
directory_path = platform_directory + '/' + folder_project + '/' + folder_ready+ '/' + folder_year +'/'+school_name + '/' + folder_export + '/'

folder_path1 = directory_path + data_for2 + '/'  #+ time_series_resample + '_'  # replace with the folder path
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith('all.xlsx') and f.startswith(survey_resample + '_' + school_name)] # get list of csv files
dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_excel(file_path) # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list

# concatenate dataframes in the list into a single dataframe
combined2 = pd.concat(dfs1, ignore_index=True)
combined2['temp_coldhot'] = combined2['temp_coldhot'].fillna(0)
#we convert time zone of dataframe, from GMT to Local
time_zone_converter(combined2, 'UTC', 'Europe/Oslo')
datetime_indexer_noindex(combined2)
print(folder_path1)
#filterized a 'lazy responses', if a response contains 0 (skipped answer) value, it would be eliminated
#min_feel = 1
#combined2 = combined2[(combined2['feel_air']>=min_feel) & (combined2['feel_temp']>=min_feel) & (combined2['feel_health']>=min_feel) & (combined2['feel_bright']>=min_feel) & (combined2['feel_noise']>=min_feel)]

In [669]:
#put spesific Directory and File Name.
directory_path = platform_directory + '/' + folder_project + '/' + folder_ready+ '/' + folder_year +'/'+school_name + '/'

#specific for each season
combined2['date'] = combined2['datetime'].dt.strftime('%Y-%m-%d')
combined2['time'] = combined2['datetime'].dt.strftime('%H:%M')
name_date = datetime.strptime((combined2[date_name].iloc[1]),'%Y-%m-%d').strftime('%Y%m%d')
filename = school_name+'_' + name_date + '_combined all classes.csv'
filenamexl = school_name+'_' + name_date + '_combined all classes.xlsx'

# #special for all years
# filename = school_name+'_' + '_alldate.csv'
# filenamexl = school_name+'_' + '_alldate.xlsx'

# save dataframe to CSV file in specified directory
directory_export = directory_path + folder_export +'/' + data_for2 + '/'  + survey_resample
combined2.to_excel( directory_export + '_2_' + filenamexl, index=False)
combined2.to_csv( directory_export + '_2_' + filename, index=False)
combined2.describe().to_excel( directory_export + '_descstat_2_' + school_name + '_' + name_date + '.xlsx', index=True)

In [ ]:
print(combined2['room_id'].unique())
print(combined2['school_id'].unique())
print(directory_export + '_2_' + filenamexl)

In [ ]:
combined2.iloc[:,23:45].describe().iloc[1:].round(1)

## PIVOT all schools

In [58]:
school_name = 'all schools'
folder_export = 'export'
data_for4 = 'combined longitudinal'
last_file = 'alldate.xlsx'

directory_path = platform_directory + '/' + folder_project + '/' + folder_ready+ '/' + folder_year +'/'+school_name + '/' + folder_export + '/'

folder_path1 = directory_path + data_for4 + '/'  #+ time_series_resample + '_'  # replace with the folder path


In [ ]:
folder_path1

In [60]:
csv_files1 = [f for f in os.listdir(folder_path1) if f.endswith(last_file) and f.startswith(survey_resample )] # get list of csv files
dfs1 = [] # list to store dataframes

for file in csv_files1:
    file_path = os.path.join(folder_path1, file) # get full path of csv file
    df = pd.read_excel(file_path) # read csv file into a dataframe
    dfs1.append(df) # append dataframe to list
combined4 = pd.concat(dfs1, ignore_index=True)

In [64]:
combined4.to_excel( folder_path1 + survey_resample + '_all schools_' + last_file , index=False)
combined4.to_csv( folder_path1 + survey_resample + '_all schools_' + 'alldate.csv' , index=False)
